In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0, '/content/drive/MyDrive/FL2022/Stuff/ULTIMATE')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import torch.utils.tensorboard as tb
from options import args_parser
from utils import get_dataset, exp_details, plotFigure
from model_RN50 import ResNet50
import os
os.chdir("/content/drive/MyDrive/FL2022/Centralized/resultsHP")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
args=args_parser()
comment= f'lr={args.lr} batch_size={args.local_bs} wd={args.wd} optimizer={args.optimizer} norm={args.norm}'
print(comment)
tb = SummaryWriter(comment=comment)

In [ ]:
train_set, _, _ = get_dataset(args)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=args.local_bs, shuffle=True, num_workers=2) 

In [ ]:
# Model
resnet = ResNet50(args.norm)
resnet = resnet.to(device)

In [ ]:
# Optimizer
criterion = nn.CrossEntropyLoss()
if (args.optimizer == "adam"):
  optimizer = optim.Adam(resnet.parameters(), lr=args.lr) #ADAM optimizer
else: #SGD optimizer
  optimizer = optim.SGD(resnet.parameters(), lr=args.lr,
                        momentum=args.momentum, weight_decay=args.wd) 

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [ ]:
# Train
def train(epoch):
    print('\nEpoch: %d' % epoch)
    resnet.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (X_train, y_train) in enumerate(train_loader):
        X_train, y_train = X_train.to(device), y_train.to(device)
        optimizer.zero_grad()
        y_pred = resnet(X_train)
        loss = criterion(y_pred, y_train)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = y_pred.max(1)
        total += y_train.size(0)
        correct += predicted.eq(y_train).sum().item()
    
    train_loss_avg = train_loss/(batch_idx+1)
    train_acc_avg = correct/total
    print(f'TRAIN -> Epoch: {epoch:2} loss: {train_loss_avg:10.8f} accuracy: {100.*correct/total:7.3f}% ({correct}/{total})')

  
    return train_loss_avg, train_acc_avg

In [ ]:
for epoch in range(1,args.epochs+1):
  train_loss, train_acc = train(epoch)
  tb.add_scalar('Loss', train_loss, epoch)
  tb.add_scalar('Accuracy', train_acc, epoch)

